In [10]:
interaction_dataframe = ppi_df
columns = ['xref_A', 'xref_B']
identifier_series = pd.Series(pd.unique(interaction_dataframe[columns].values.ravel('K')))
ids = identifier_series[identifier_series.str.startswith('ensembl:')]

wa


In [18]:
from pathlib import Path
import pandas as pd
import numpy as np

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import ppi_filter
from phppipy.ppi_tools import id_mapper

In [19]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged.tsv')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
interaction_dataframe = ppi_df
columns = ['xref_A', 'xref_B']
id_mapper.check_unique_identifier(interaction_dataframe)
db_tag='refseq:'
id_abbreviation='P_REFSEQ_AC'
merged_columns = pd.unique(
        interaction_dataframe[columns].values.ravel('K'))
filepath = '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new2'
path = Path(filepath) / (
            db_tag.replace('/', '-').strip(':') + r'2uniprot.txt')
id_mapper._create_mapping_files(merged_columns, id_abbreviation, db_tag,
                                  path)


30332 unique identifiers in column xref_A did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

30332 unique identifiers in column xref_B did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

refseq: 17 identifiers found in dataset.
Created mapping file between UniProt ACs and refseq in: /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new2/refseq2uniprot.txt.



In [20]:
mapping_dict = id_mapper._create_mapping_dict(path, True)

# print(mapping_dict)

# create helper function to use in apply() acting on columns to be remapped
def lambda_helper(row_entry, mapping_dict):
    # retrieve id mapping if it exists, otherwise retain original id
    original_id = row_entry.split(':')[1]
    new_id = mapping_dict.get(original_id, None)

    # if no list is returned by dict.get(), original id is passed
    if new_id:
        # if a new id is found in the mapping_dict, return it
        # mapping_dict contains lists, so extract or join for multiple mappings
        if len(new_id) > 1:
            new_id = 'MULT_MAP'.join(new_id)
        else:  # retrieve id as string from list
            new_id = new_id[0]
    else:
        # return original id if no remapping is found
        new_id = row_entry
    return new_id

success_series = []
total_series = []
for col in columns:
    # note: use startswith to avoid mixing ensembl and embl ids...
    mapping_selection = interaction_dataframe[col].str.startswith(
        db_tag)
    print(mapping_selection.sum())
#     print(interaction_dataframe.loc[mapping_selection,col])

    # keep track of total (unique) identifier count
    total_series.append(interaction_dataframe.loc[mapping_selection,col])
#     print(total_series)

    interaction_dataframe.loc[mapping_selection, col] = \
        interaction_dataframe.loc[mapping_selection, col].apply(func=lambda_helper, args=(mapping_dict,))

    # count how many (unique) identifiers were remapped
    success_series.append(interaction_dataframe.loc[mapping_selection & ~interaction_dataframe[col].str.startswith(db_tag), col])
#     print(success_series)
    
succes_count = pd.concat(success_series).unique().size
total_count = pd.concat(total_series).unique().size

# print(pd.concat(success_series).unique())
# print(pd.concat(total_series).unique())

print(
    '{} {} out of identifiers {} were succesfully remapped to uniprot accession numbers.\n'.
    format(db_tag, succes_count, total_count))

6
94
refseq: 2 out of identifiers 17 were succesfully remapped to uniprot accession numbers.



In [37]:
col='xref_A'
interaction_dataframe.loc[mapping_selection & ~interaction_dataframe[col].str.startswith(db_tag),col]

1428         uniprotkb:Q00403
1429         uniprotkb:Q00403
1439         uniprotkb:P14859
1440         uniprotkb:P14859
1455         uniprotkb:Q9Y4K3
1456         uniprotkb:Q9Y4K3
1457         uniprotkb:Q9Y4K3
1458         uniprotkb:Q9Y4K3
1463         uniprotkb:Q15750
1467         uniprotkb:Q9NYJ8
1535         uniprotkb:Q8VDD5
1536         uniprotkb:Q8VDD5
1537         uniprotkb:Q8VDD5
1538         uniprotkb:Q8VDD5
1592         uniprotkb:Q71SY5
1594         uniprotkb:Q71SY5
1597         uniprotkb:Q71SY5
1600         uniprotkb:Q71SY5
1601         uniprotkb:Q71SY5
1637         uniprotkb:P35579
1638         uniprotkb:P35579
1763         uniprotkb:P51617
1818         uniprotkb:Q16594
1907         uniprotkb:Q8VCD5
1909         uniprotkb:Q6PGF3
1960         uniprotkb:A2AGH6
1961         uniprotkb:D9MWX4
1962         uniprotkb:D9MWX4
1964         uniprotkb:Q9CQ39
1967         uniprotkb:P49116
                 ...         
19767    uniprotkb:A0A1S5YGR5
19771    uniprotkb:A0A1S5YGR6
19775    u

In [14]:
a = pd.Series()
a

Series([], dtype: float64)

In [5]:
if Path(path).is_file():

            # create dictionaries for current identifier remapping file
            mapping_dict = id_mapper._create_mapping_dict(path, True, db_tag)

            # create helper function to use in apply() acting on columns to be remapped
            def lambda_helper(row_entry, mapping_dict):
                # retrieve id mapping if it exists, otherwise retain original id
                original_id = row_entry.split(':')[1]
                new_id = mapping_dict.get(original_id, None)

                # if no list is returned by dict.get(), original id is passed
                if new_id:
                    # if a new id is found in the mapping_dict, return it
                    # mapping_dict contains lists, so extract or join for multiple mappings
                    if len(new_id) > 1:
                        new_id = 'MULT_MAP'.join(new_id)
                    else:  # retrieve id as string from list
                        new_id = new_id[0]
                else:
                    # return original id if no remapping is found
                    new_id = original_id
                return new_id

            # go through all IDs in the supplied columns and remap them if needed
            succes_count = 0
            total_count = 0
            for col in columns:
                # note: use startswith to avoid mixing ensembl and embl ids...
                mapping_selection = interaction_dataframe[col].str.startswith(
                    db_tag)
                interaction_dataframe.loc[mapping_selection, col] = \
                    interaction_dataframe.loc[mapping_selection, col].apply(func=lambda_helper, args=(mapping_dict,))
                succes_count += mapping_selection.sum() - interaction_dataframe[col].str.startswith(db_tag).sum()
                total_count += mapping_selection.sum()

            print(
                '{} {} out of identifiers {} were succesfully remapped to uniprot accession numbers.\n'.
                format(db_tag, succes_count, total_count))

refseq: 100 out of identifiers 100 were succesfully remapped to uniprot accession numbers.



In [23]:
ppi_df.loc[ppi_df.publication == 'doi:10.1038/nature11288|pubmed:22810586|imex:IM-25307']

,aliases_A,aliases_B,alt_identifiers_A,alt_identifiers_B,author,confidence_score,detection_method,display_id_A,display_id_B,interaction_identifiers,...,publication,source_database_ids,taxid_A,taxid_B,xref_A,xref_B,inter-intra-species,inter-intra-pathogen,taxid_A_name,taxid_B_name
4149,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:bnl2a_ebvb9(display_long)|uniprotkb:BNL...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-9346744|uniprotkb:Q04361|uniprotkb:...,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11725101|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P16070,uniprotkb:P0C739,inter,inter,Homo sapiens,Human herpesvirus 4 strain B95-8
4150,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:teg7_ebvb9(display_long)|uniprotkb:BSRF...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-2621334|uniprotkb:P03194|uniprotkb:...,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11725356|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P16070,uniprotkb:P0CK49,inter,inter,Homo sapiens,Human herpesvirus 4 strain B95-8
4151,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:balf1_ebvb9(display_long)|uniprotkb:BAL...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-2621293|uniprotkb:P03229|uniprotkb:...,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11732874|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P16070,uniprotkb:P0CK58,inter,inter,Homo sapiens,Human herpesvirus 4 strain B95-8
4152,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:lmp2_ebvb9(display_long)|uniprotkb:LMP2...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-7181113|uniprotkb:Q8AZK9|intact:MIN...,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11733017|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P16070,uniprotkb:P13285,inter,inter,Homo sapiens,Human herpesvirus 4 strain B95-8
4153,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:ul92_ebvb9(display_long)|uniprotkb:BDLF...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-2622323|uniprotkb:P03223|uniprotkb:...,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11725466|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P16070,uniprotkb:P0CK56,inter,inter,Homo sapiens,Human herpesvirus 4 strain B95-8
4154,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:ear_ebvb9(display_long)|uniprotkb:BHRF1...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-1207659|uniprotkb:Q777H0,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11721938|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P16070,uniprotkb:P03182,inter,inter,Homo sapiens,Human herpesvirus 4 strain B95-8
4155,psi-mi:cd44_human(display_long)|uniprotkb:Epic...,psi-mi:q2mg95_ebvb9(display_long)|uniprotkb:BV...,intact:EBI-490245|uniprotkb:P22511|uniprotkb:Q...,intact:EBI-11733646,Rozenblatt-Rosen et al. (2012),intact-miscore:0.35,psi-mi:MI:0676(tandem affinity purification),NaN,NaN,intact:EBI-11733653|psi-mi:MI:0471|imex:IM-253...,...,doi:10.1038/nature11288|pubmed:22810586|imex:I...,psi-mi:MI:0471(MINT),taxid:9606,taxid:10377,uniprotkb:P1

In [62]:
from pathlib import Path
import pandas as pd
import numpy as np

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import ppi_filter
from phppipy.ppi_tools import id_mapper

In [63]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged.tsv')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)

In [64]:
columns = ['xref_A','xref_B']

In [65]:
ppi_df['unique'] = ppi_df.apply(lambda x: '%'.join(sorted([x['xref_A'], x['xref_B']])), axis=1)

In [66]:
xref_partners_sorted_array = np.sort(np.stack((ppi_df.xref_A, ppi_df.xref_B), axis=1), axis=1)
xref_partners_sorted_array
xref_partners_df = pd.DataFrame(xref_partners_sorted_array, columns=['A', 'B'])
ppi_df['xref_partners_sorted'] = xref_partners_df['A'] + '%' + xref_partners_df['B']

In [67]:
all(ppi_df.xref_partners_sorted == ppi_df.unique)

True

In [68]:
print(np.sum(ppi_df.duplicated(subset=['xref_partners_sorted'])))
print('\nNumber of unique interactions per raw data set')
print(ppi_df.groupby('origin')['xref_partners_sorted'].nunique())
print(pd.unique(ppi_df.xref_partners_sorted).size)
print(ppi_df.xref_partners_sorted.unique().size)
print(ppi_df.xref_partners_sorted.size)
print(18490+11842)

ppi_df['original_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])

18490

Number of unique interactions per raw data set
origin
BIOGRID-ALL-3.4.160.mitab        1186
hpidb2-19.02.2018.mitab          7609
intact-virus-22.03.2018.mitab    4019
phi_data.csv                     6740
virhostnet-01.2018.mitab         5775
Name: xref_partners_sorted, dtype: int64
11842
11842
30332
30332


In [69]:
id_mapper.check_unique_identifier(ppi_df)

30332 unique identifiers in column xref_A did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

30332 unique identifiers in column xref_B did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.



In [70]:
xref_partners_sorted_array = np.sort(np.stack((ppi_df.xref_A, ppi_df.xref_B), axis=1), axis=1)
xref_partners_df = pd.DataFrame(xref_partners_sorted_array, columns=['A', 'B'])
ppi_df['xref_partners_sorted'] = xref_partners_df['A'] + '%' + xref_partners_df['B']
print(np.sum(ppi_df.duplicated(subset=['xref_partners_sorted'])))
print('\nNumber of unique interactions per raw data set')
print(ppi_df.groupby('origin')['xref_partners_sorted'].nunique())
print(pd.unique(ppi_df.xref_partners_sorted).size)
print(ppi_df.xref_partners_sorted.unique().size)
print(ppi_df.xref_partners_sorted.size)
print(18490+11842)
ppi_df['checker_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])

18796

Number of unique interactions per raw data set
origin
BIOGRID-ALL-3.4.160.mitab        1186
hpidb2-19.02.2018.mitab          7311
intact-virus-22.03.2018.mitab    4019
phi_data.csv                     6740
virhostnet-01.2018.mitab         5775
Name: xref_partners_sorted, dtype: int64
11536
11536
30332
30332


In [71]:
id_mapper.map2uniprot(ppi_df, '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')


No ddbj/embl/genbank: identifiers found in dataset.

No ensembl: identifiers found in dataset.

No ensemblgenomes: identifiers found in dataset.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/entrez gene-locuslink2uniprot.txt mapping file already exists, not regenerating...

entrez gene/locuslink: 1080 out of identifiers 1111 were succesfully remapped to UniProt accession numbers.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/refseq2uniprot.txt mapping file already exists, not regenerating...

refseq: 2 out of identifiers 17 were succesfully remapped to UniProt accession numbers.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/dip2uniprot.txt mapping file already exists, not regenerating...

dip: 0 out of identifiers 0 were succesfully remapped to UniProt accession numbers.

19 identifiers were be remapped to multiple U

In [99]:
print(ppi_df.groupby('origin')['xref_partners_sorted'].size())

origin
BIOGRID-ALL-3.4.160.mitab        1427
hpidb2-19.02.2018.mitab          9129
intact-virus-22.03.2018.mitab    4529
phi_data.csv                     8118
virhostnet-01.2018.mitab         7129
Name: xref_partners_sorted, dtype: int64


In [72]:
xref_partners_sorted_array = np.sort(np.stack((ppi_df.xref_A, ppi_df.xref_B), axis=1), axis=1)
xref_partners_df = pd.DataFrame(xref_partners_sorted_array, columns=['A', 'B'])
ppi_df['xref_partners_sorted'] = xref_partners_df['A'] + '%' + xref_partners_df['B']
print(np.sum(ppi_df.duplicated(subset=['xref_partners_sorted'])))
print('\nNumber of unique interactions per raw data set')
print(ppi_df.groupby('origin')['xref_partners_sorted'].nunique())
print(pd.unique(ppi_df.xref_partners_sorted).size)
print(ppi_df.xref_partners_sorted.unique().size)
print(ppi_df.xref_partners_sorted.size)
print(18490+11842)
ppi_df['remap_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])

19271

Number of unique interactions per raw data set
origin
BIOGRID-ALL-3.4.160.mitab        1186
hpidb2-19.02.2018.mitab          7210
intact-virus-22.03.2018.mitab    4019
phi_data.csv                     6740
virhostnet-01.2018.mitab         5775
Name: xref_partners_sorted, dtype: int64
11061
11061
30332
30332


In [85]:
ppi_df.loc[ (ppi_df.checker_unique == True) & (ppi_df.remap_unique == False), ['xref_partners_sorted', 'original_unique', 'remap_unique', 'checker_unique']].size

1900

In [92]:
ppi_df.loc[ (ppi_df.checker_unique == True) & (ppi_df.remap_unique == False)].groupby('origin').size()

origin
hpidb2-19.02.2018.mitab          279
intact-virus-22.03.2018.mitab      2
phi_data.csv                     192
virhostnet-01.2018.mitab           2
dtype: int64

In [81]:
ppi_df.loc[ (ppi_df.original_unique == True) & (ppi_df.checker_unique == False), ['xref_partners_sorted', 'original_unique', 'remap_unique', 'checker_unique']].size

1224

In [91]:
ppi_df.loc[ (ppi_df.original_unique == True) & (ppi_df.checker_unique == False)].groupby('origin').size()

origin
hpidb2-19.02.2018.mitab    298
phi_data.csv                 8
dtype: int64

In [83]:
ppi_df.loc[ (ppi_df.original_unique == True) & (ppi_df.remap_unique == False), ['xref_partners_sorted', 'original_unique', 'remap_unique', 'checker_unique']].size

3124

In [93]:
ppi_df.loc[ (ppi_df.original_unique == True) & (ppi_df.remap_unique == False)].groupby('origin').size()

origin
hpidb2-19.02.2018.mitab          577
intact-virus-22.03.2018.mitab      2
phi_data.csv                     200
virhostnet-01.2018.mitab           2
dtype: int64

In [108]:
ppi_df.duplicated(subset=['xref_partners_sorted', 'publication']).size

30332

In [109]:
print('Number of unique interactions per raw dataset:')
print(ppi_df.groupby('origin')['xref_partners_sorted'].nunique())
print('\nTotal dataset size:')
print(ppi_df.groupby('origin')['xref_partners_sorted'].size())
print('\nTotal number of unique interactions out of {}'.format(ppi_df.size))
print(np.sum(~ppi_df.duplicated(subset=['xref_partners_sorted'])))
print('\nTotal number of unique interactions out of {}, where publications are considered unique:'.format(ppi_df.size))
print(np.sum(~ppi_df.duplicated(subset=['xref_partners_sorted', 'publication'])))

Number of unique interactions per raw dataset:
origin
BIOGRID-ALL-3.4.160.mitab        1186
hpidb2-19.02.2018.mitab          7210
intact-virus-22.03.2018.mitab    4019
phi_data.csv                     6740
virhostnet-01.2018.mitab         5775
Name: xref_partners_sorted, dtype: int64

Total dataset size:
origin
BIOGRID-ALL-3.4.160.mitab        1427
hpidb2-19.02.2018.mitab          9129
intact-virus-22.03.2018.mitab    4529
phi_data.csv                     8118
virhostnet-01.2018.mitab         7129
Name: xref_partners_sorted, dtype: int64

Total number of unique interactions out of 849296
11061

Total number of unique interactions out of 849296, where publications are considered unique:
15003


In [112]:
np.unique(ppi_df['origin'].values)

array(['BIOGRID-ALL-3.4.160.mitab', 'hpidb2-19.02.2018.mitab',
       'intact-virus-22.03.2018.mitab', 'phi_data.csv',
       'virhostnet-01.2018.mitab'], dtype=object)

In [113]:
ppi_df.drop_duplicates(subset=['xref_partners_sorted'], keep='first').groupby('origin').size()

origin
BIOGRID-ALL-3.4.160.mitab        1186
hpidb2-19.02.2018.mitab          6444
intact-virus-22.03.2018.mitab    1095
phi_data.csv                      436
virhostnet-01.2018.mitab         1900
dtype: int64

In [116]:
ppi_df['origin'] = pd.Categorical(ppi_df['origin'], ['intact-virus-22.03.2018.mitab',
                                                     'BIOGRID-ALL-3.4.160.mitab',
                                                     'hpidb2-19.02.2018.mitab',
                                                     'virhostnet-01.2018.mitab',
                                                     'phi_data.csv'])
ppi_df.sort_values(by='origin', ascending=True, inplace=True)
ppi_df.drop_duplicates(subset=['xref_partners_sorted'], keep='first').groupby('origin').size()


origin
intact-virus-22.03.2018.mitab    4019
BIOGRID-ALL-3.4.160.mitab        1166
hpidb2-19.02.2018.mitab          3540
virhostnet-01.2018.mitab         1900
phi_data.csv                      436
dtype: int64

In [7]:
ppi_df.loc[ppi_df[columns[0]].str.contains('MULT_MAP') | ppi_df[columns[1]].str.contains('MULT_MAP')].shape[0]

429

In [118]:
columns = ['xref_A', 'xref_B']
interaction_dataframe = ppi_df
selection = (~interaction_dataframe[columns[0]].str.contains('MULT_MAP') &
                 ~interaction_dataframe[columns[1]].str.contains('MULT_MAP'))

print(
    'Omitted {} PPIs due to the existance of multiple mappings.\n'.format(
        np.sum(~selection)))

Omitted 429 PPIs due to the existance of multiple mappings.



In [120]:
interaction_dataframe.loc[selection].apply(lambda x: x).size

837284

In [125]:
ppi_df['origin'] = pd.Categorical(ppi_df['origin'], ['intact-virus-22.03.2018.mitab',
                                                     'BIOGRID-ALL-3.4.160.mitab',
                                                     'hpidb2-19.02.2018.mitab',
                                                     'virhostnet-01.2018.mitab',
                                                     'phi_data.csv'])
ppi_df.sort_values(by='origin', ascending=True, inplace=True)
size = ppi_df.shape[0]
ppi_df = ppi_df.drop_duplicates(subset=['xref_partners_sorted'], keep='first')
ppi_df = ppi_df.reset_index(drop=True)
print('All duplicate interactions were removed, leaving {} out of {} PPIs.'.format(ppi_df.shape[0], size))

All duplicate interactions were removed, leaving 11061 out of 30332 PPIs.


In [2]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
id_mapper.map2uniprot(ppi_df, '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')


No ddbj/embl/genbank: identifiers found in dataset.

No ensembl: identifiers found in dataset.

No ensemblgenomes: identifiers found in dataset.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/entrez gene-locuslink2uniprot.txt mapping file already exists, not regenerating...

entrez gene/locuslink: 1080 out of identifiers 1111 were succesfully remapped to uniprot accession numbers.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/refseq2uniprot.txt mapping file already exists, not regenerating...

refseq: 0 out of identifiers 12 were succesfully remapped to uniprot accession numbers.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/dip2uniprot.txt mapping file already exists, not regenerating...

dip: 0 out of identifiers 359 were succesfully remapped to uniprot accession numbers.

19 identifiers were be remapped to multiple

In [2]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
id_mapper.map2uniprot(ppi_df, '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')


No ddbj/embl/genbank: identifiers found in dataset.

No ensembl: identifiers found in dataset.

No ensemblgenomes: identifiers found in dataset.

/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/entrez gene-locuslink2uniprot.txt mapping file already exists, not regenerating...

entrez gene/locuslink: 1596 out of identifiers 1596 were succesfully remapped to uniprot accession numbers.

refseq: 12 identifiers found in dataset.
Created mapping file between UniProt ACs and refseq in: /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/refseq2uniprot.txt.

refseq: 13 out of identifiers 13 were succesfully remapped to uniprot accession numbers.

dip: 359 identifiers found in dataset.
Created mapping file between UniProt ACs and dip in: /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/dip2uniprot.txt.

dip: 359 out of identifiers 359 were

In [2]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
id_mapper.map2uniprot(ppi_df, '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')


ddbj/embl/genbank: 0 identifiers found in dataset.
No succesful mappings found for ddbj/embl/genbank.

ensembl: 0 identifiers found in dataset.
No succesful mappings found for ensembl.

ensemblgenomes: 0 identifiers found in dataset.
No succesful mappings found for ensemblgenomes.

entrez gene/locuslink: 1111 identifiers found in dataset.
Created mapping files between UniProt ACs and entrez gene/locuslink in the data sets.
Stored in /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/entrez gene-locuslink2uniprot.txt.

entrez gene/locuslink: 668 out of 2164 succesfully mapped identifiers were unreviewed and excluded from the dataset.

entrez gene/locuslink: 4270 out of identifiers 4270 were succesfully remapped to uniprot accession numbers.

refseq: 12 identifiers found in dataset.
No succesful mappings found for refseq.

dip: 359 identifiers found in dataset.
Created mapping files between UniProt ACs and dip in the data sets.
Stored i

In [14]:

ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
id_mapper.map2uniprot(ppi_df, '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')


entrez gene/locuslink: 1082 identifiers were remapped to a reviewed uniprot accession number.

entrez gene/locuslink: 668 identifiers were not remapped to an unreviewed uniprot accession number.

dip: 299 identifiers were remapped to a reviewed uniprot accession number.

dip: 49 identifiers were not remapped to an unreviewed uniprot accession number.

19 identifiers were be remapped to multiple uniprot accession numbers.



In [8]:
interaction_dataframe = ppi_df
columns = ['xref_A', 'xref_B']

merged_columns = pd.unique(
        interaction_dataframe[columns].values.ravel('K'))

present_identifiers = id_mapper._extract_identifiers(merged_columns)

identifier_dict = {
    'ddbj/embl/genbank:': 'EMBL_ID',
    'ensembl:': 'ENSEMBL_ID',
    'ensemblgenomes:': 'ENSEMBLGENOME_ID',
    'entrez gene/locuslink:': 'P_ENTREZGENEID',
    'refseq:': 'P_REFSEQ_AC',
    'dip:': 'DIP_ID'
}

# NOTE: UniProt REST API does not support intact EBI:identifiers.



In [9]:
print(present_identifiers)
for i in present_identifiers:
    if i + ':' not in identifier_dict:
        print(
            'WARNING: interaction dataset contains "{}" entries, which could not be remapped to UniProt AC (check all possible mappings at https://www.uniprot.org/help/api_idmapping .'.
            format(i))

['entrez gene/locuslink', 'dip', 'intact', 'refseq']


In [10]:
id_mapper.check_unique_identifier(interaction_dataframe)

30332 unique identifiers in column xref_A did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

30332 unique identifiers in column xref_B did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.



In [9]:
filepath = '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new'
for db_tag, id_abbreviation in identifier_dict.items():
    # Define filepath (without forward slashes)
    path = Path(filepath) / (
        db_tag.replace('/', '-').strip(':') + r'2uniprot.txt')
    # path.parent.mkdir(parents=True, exist_ok=True) # moved to create_mapping_files function

    # Skip if file already exists
    # TODO: add options to force re-run
    if not Path(path).is_file():
        id_mapper._create_mapping_files(merged_columns, id_abbreviation, db_tag,
                              path)
    with path.open() as mapping_file:
        # TODO: possible to keep unreviewed identifiers? Will result in many more 1:n mappings...

        # Store mapping into dictionary where non-UniProt keys map to lists of UniProtACs.
        mapping_dict = {}
        reviewed_mapping_dict = {}
        unreviewed_mapping_dict = {}

        for line in mapping_file:
            # [0] contains the original identifier
            # [1] is an empty element in most mapping files
            # [2] contains the UniProt AC
            split_line = line.split('\t')
            from_id = split_line[0]
            uniprot_ac = 'uniprotkb:' + split_line[2]

            if '\treviewed' in line:
                # assert from_id not in mapping_dict, 'WARNING: the remapping file {} contains duplicates!'.format(path.resolve())
                if from_id not in mapping_dict:
                    reviewed_mapping_dict[from_id] = [uniprot_ac]
                else:
                    # even when only selecting reviewed proteins, there are still 1:n mappings
                    # e.g. http://www.uniprot.org/uniprot/?query=yourlist:M2017111583C3DD8CE55183C76102DC5D3A26728BF70646W&sort=yourlist:M2017111583C3DD8CE55183C76102DC5D3A26728BF70646W&columns=yourlist%28M2017111583C3DD8CE55183C76102DC5D3A26728BF70646W%29,isomap%28M2017111583C3DD8CE55183C76102DC5D3A26728BF70646W%29,id,entry+name,reviewed,protein+names,genes,organism,length
                    reviewed_mapping_dict[from_id].append(uniprot_ac)

            elif '\tunreviewed' in line:
                # assert from_id not in unreviewed_mapping_dict, 'WARNING: the remapping file {} contains duplicates!'.format(path.resolve())
                if from_id not in unreviewed_mapping_dict:
                    unreviewed_mapping_dict[from_id] = [uniprot_ac]
                else:
                    unreviewed_mapping_dict[from_id].append(uniprot_ac)


FileNotFoundError: [Errno 2] No such file or directory: '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/ddbj-embl-genbank2uniprot.txt'

In [23]:
interaction_dataframe.loc[1:10, 'xref_A'].apply(type)

1     <class 'str'>
2     <class 'str'>
3     <class 'str'>
4     <class 'str'>
5     <class 'str'>
6     <class 'str'>
7     <class 'str'>
8     <class 'str'>
9     <class 'str'>
10    <class 'str'>
Name: xref_A, dtype: object

In [13]:
interaction_dataframe['xref_A'].str.startswith('refseq').sum()

6

In [9]:
pd.Series(
        pd.unique(ppi_df[['xref_A','xref_B']].values.ravel(
            'K'))).str.contains('MULT_MAP')

19

In [10]:
ppi_df.loc[ppi_df.xref_A.str.contains('MULT_MAP') | ppi_df.xref_B.str.contains('MULT_MAP')]

,aliases_A,aliases_B,alt_identifiers_A,alt_identifiers_B,author,confidence_score,detection_method,display_id_A,display_id_B,interaction_identifiers,...,publication,source_database_ids,taxid_A,taxid_B,xref_A,xref_B,inter-intra-species,inter-intra-pathogen,taxid_A_name,taxid_B_name
53,-,entrez gene/locuslink:APG-2(gene name synonym)...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:109540|entrez gene/locuslink:HSPA4,Forsman A (2008),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:592318,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P34932,inter,inter,Human gammaherpesvirus 4,Homo sapiens
54,-,entrez gene/locuslink:HEL-33(gene name synonym...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:109544|entrez gene/locuslink:HSPA8,Forsman A (2008),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:592319,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P11142,inter,inter,Human gammaherpesvirus 4,Homo sapiens
55,-,entrez gene/locuslink:K-ALPHA-1(gene name syno...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:115651|entrez gene/locuslink:TUBA1B,Forsman A (2008),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:592320,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P68363,inter,inter,Human gammaherpesvirus 4,Homo sapiens
56,-,entrez gene/locuslink:CDCBM6(gene name synonym...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:128444|entrez gene/locuslink:TUBB|entr...,Forsman A (2008),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:592321,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P07437,inter,inter,Human gammaherpesvirus 4,Homo sapiens
57,-,entrez gene/locuslink:APG-2(gene name synonym)...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:109540|entrez gene/locuslink:HSPA4,Forsman A (2008),-,"psi-mi:""MI:0401""(biochemical)",NaN,NaN,biogrid:592322,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P34932,inter,inter,Human gammaherpesvirus 4,Homo sapiens
58,-,entrez gene/locuslink:HEL-33(gene name synonym...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:109544|entrez gene/locuslink:HSPA8,Forsman A (2008),-,"psi-mi:""MI:0401""(biochemical)",NaN,NaN,biogrid:592323,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P11142,inter,inter,Human gammaherpesvirus 4,Homo sapiens
59,-,entrez gene/locuslink:K-ALPHA-1(gene name syno...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:115651|entrez gene/locuslink:TUBA1B,Forsman A (2008),-,"psi-mi:""MI:0401""(biochemical)",NaN,NaN,biogrid:592324,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P68363,inter,inter,Human gammaherpesvirus 4,Homo sapiens
60,-,entrez gene/locuslink:CDCBM6(gene name synonym...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:128444|entrez gene/locuslink:TUBB|entr...,Forsman A (2008),-,"psi-mi:""MI:0401""(biochemical)",NaN,NaN,biogrid:592325,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:P07437,inter,inter,Human gammaherpesvirus 4,Homo sapiens
61,-,entrez gene/locuslink:BAG-2(gene name synonym)...,biogrid:971788|entrez gene/locuslink:EBNA-LP,biogrid:114908|entrez gene/locuslink:BAG2|entr...,Forsman A (2008),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:592326,...,pubmed:18457437,"psi-mi:""MI:0463""(biogrid)",taxid:10376,taxid:9606,uniprotkb:P0C732MULT_MAPuniprotkb:Q8AZK7,uniprotkb:O95816,inter,inter,Human gammaherpesvirus 4,Homo sapi

In [35]:
from pathlib import Path
import pandas as pd
import numpy as np

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import ppi_filter
from phppipy.ppi_tools import id_mapper

ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged.tsv')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
interaction_dataframe = ppi_df
columns = ['xref_A', 'xref_B']
out_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')


In [36]:
from pathlib import Path
import pandas as pd
import numpy as np

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import ppi_filter
from phppipy.ppi_tools import id_mapper

ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged.tsv')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
interaction_dataframe = ppi_df
columns = ['xref_A', 'xref_B']
out_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')

input_dir = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/taxonid')
pathogen_taxonid_path = [i for i in input_dir.glob('*child*')]
associated_taxonid_path = [i for i in input_dir.glob('*interacting*')]
try:
    assert len(pathogen_taxonid_path) == 1
    assert len(associated_taxonid_path) == 1
except AssertionError:
    print(
        'WARNING: multiple possible taxonID files were found in the directory {}'.
        format(input_dir))
    print(
        'Please run the pathogen_selection.py script again and verify that only two files are being created in a pathogen-specific directory, e.g. 10292/10292-child-taxids.txt and 10292/associated-taxids.txt'
    )


pathogen_taxonids = taxonid.read_taxids(pathogen_taxonid_path[0])
host_taxonids = taxonid.read_taxids(associated_taxonid_path[0])
print('TaxonIDs were read from {} and {}\n'.format(pathogen_taxonid_path[0],
                                                   associated_taxonid_path[0]))

print(host_taxonids[0])


ppi_filter.annotate_inter_intra_species(ppi_df)
ppi_filter.annotate_inter_intra_pathogen(ppi_df, pathogen_taxonids)
# assert all(temp_intra_species == ppi_df['inter-intra-species'])
# assert all(temp_intra_pathogen == ppi_df['inter-intra-pathogen'])
ppi_df = ppi_df.loc[(ppi_df['inter-intra-species'] != 'intra')
                    & (ppi_df['inter-intra-pathogen'] != 'intra')]
ppi_df = ppi_df.reset_index(drop=True)
ppi_df.drop(
    ['inter-intra-species', 'inter-intra-pathogen'], axis=1, inplace=True)
print('All intra-species and intra-pathogen interactions were omitted.')
# check for intra-hosts cross species
assert any(
    ppi_df.taxid_A.isin(host_taxonids) & ppi_df.taxid_B.isin(host_taxonids)
) is False

TaxonIDs were read from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/taxonid/10292-child-taxids.txt and /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/taxonid/interacting-taxids.txt

taxid:9534
All intra-species and intra-pathogen interactions were omitted.


In [37]:
# create unique identifier by combining xrefs
ppi_filter.unique_identifier(ppi_df)
ppi_df['original_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])

# fix bad xref identifiers (i.e. containing more than a single entry)
id_mapper.check_unique_identifier(ppi_df)

# re-create unique identifier by combining xrefs
ppi_filter.unique_identifier(ppi_df)
ppi_df['checker_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])
print(
    'By removing/fixing protein identifiers that consisted of multiple entries, the following duplicates were introduced in the dataset:\n'
)
print(ppi_df.loc[(ppi_df.original_unique)
                 & ~(ppi_df.checker_unique)].groupby('origin').size(),'\n')

# create UniProt mapping files and remap identifiers
out_mappings = out_path / 'mapping'
out_mappings.parent.mkdir(parents=True, exist_ok=True)
id_mapper.map2uniprot(ppi_df, out_mappings, reviewed_only=True)

# remove multiple mappings
ppi_df = id_mapper.remove_mult(ppi_df)

# re-create unique identifier by combining xrefs
ppi_filter.unique_identifier(ppi_df)
ppi_df['remap_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])
print(
    'The act of remapping protein identifiers to UniProt AC, introduced the following duplicates:\n'
)
print(ppi_df.loc[(ppi_df.checker_unique)
                 & ~(ppi_df.remap_unique)].groupby('origin').size(),'\n')

# check if remapping creates new unique values (should never happen)
assert ppi_df.loc[
    ~(ppi_df.original_unique) & (ppi_df.remap_unique), [
        'xref_partners_sorted', 'original_unique', 'remap_unique',
        'checker_unique'
    ]].shape[0] == 0

# report information about duplicates
print('Number of unique interactions per raw dataset:')
print(ppi_df.groupby('origin')['xref_partners_sorted'].nunique(),'\n')
print('Total dataset size:')
print(ppi_df.groupby('origin')['xref_partners_sorted'].size(),'\n')
print('Total number of unique interactions out of {}'.format(
    ppi_df.shape[0]))
print(np.sum(~ppi_df.duplicated(subset=['xref_partners_sorted'])),'\n')
print(
    'Total number of unique interactions out of {}, where publications are considered unique:'.
    format(ppi_df.shape[0]))
print(
    np.sum(~ppi_df.duplicated(subset=['xref_partners_sorted', 'publication'])),'\n')
ppi_df.drop(
    ['original_unique', 'checker_unique', 'remap_unique'],
    axis=1,
    inplace=True)

26273 unique identifiers in column xref_A did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

26273 unique identifiers in column xref_B did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

By removing/fixing protein identifiers that consisted of multiple entries, the following duplicates were introduced in the dataset:

origin
hpidb2-19.02.2018.mitab    298
phi_data.csv                 8
dtype: int64 


No ddbj/embl/genbank: identifiers found in dataset.

No ensembl: identifiers found in dataset.

No ensemblgenomes: identifiers found in dataset.

entrez gene/locuslink: 1109 identifiers found in dataset.
Created mapping file between UniProt ACs and entrez gene/locuslink in: /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new/mapping/entrez gene-locuslink2uniprot.txt.

entrez gene/locuslink: 1078 out of identifiers 1109 were succesfully rema

In [38]:
# remove duplicates
# first sort on dataframe priority
ppi_df['origin'] = pd.Categorical(ppi_df['origin'], [
    'intact-virus-22.03.2018.mitab', 'BIOGRID-ALL-3.4.160.mitab',
    'hpidb2-19.02.2018.mitab', 'virhostnet-01.2018.mitab', 'phi_data.csv'
])
ppi_df.sort_values(by='origin', ascending=True, inplace=True)
size = ppi_df.shape[0]
ppi_df = ppi_df.drop_duplicates(subset=['xref_partners_sorted'], keep='first')
ppi_df = ppi_df.reset_index(drop=True)
print('All duplicate interactions were removed, leaving {} out of {} PPIs.\n'.
      format(ppi_df.shape[0], size))

# remove non-UniProt identifiers
size = ppi_df.shape[0]
ppi_df = ppi_df.loc[(ppi_df.xref_A.str.contains('uniprot'))
                    & (ppi_df.xref_B.str.contains('uniprot'))]
ppi_df = ppi_df.reset_index(drop=True)
print('Omitted {} non-UniProt AC entries, leaving {} PPIs.'.format(
    size - ppi_df.shape[0], ppi_df.shape[0]))

All duplicate interactions were removed, leaving 7893 out of 25844 PPIs.

Omitted 142 non-UniProt AC entries, leaving 7751 PPIs.


In [42]:
from pathlib import Path
import pandas as pd
import numpy as np

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import ppi_filter
from phppipy.ppi_tools import id_mapper

ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/10292-ppi-merged.tsv')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
interaction_dataframe = ppi_df
columns = ['xref_A', 'xref_B']
out_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/mappings-new')

input_dir = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/taxonid')
pathogen_taxonid_path = [i for i in input_dir.glob('*child*')]
associated_taxonid_path = [i for i in input_dir.glob('*interacting*')]
try:
    assert len(pathogen_taxonid_path) == 1
    assert len(associated_taxonid_path) == 1
except AssertionError:
    print(
        'WARNING: multiple possible taxonID files were found in the directory {}'.
        format(input_dir))
    print(
        'Please run the pathogen_selection.py script again and verify that only two files are being created in a pathogen-specific directory, e.g. 10292/10292-child-taxids.txt and 10292/associated-taxids.txt'
    )


pathogen_taxonids = taxonid.read_taxids(pathogen_taxonid_path[0])
host_taxonids = taxonid.read_taxids(associated_taxonid_path[0])
print('TaxonIDs were read from {} and {}\n'.format(pathogen_taxonid_path[0],
                                                   associated_taxonid_path[0]))

print(host_taxonids[0])


ppi_filter.annotate_inter_intra_species(ppi_df)
ppi_filter.annotate_inter_intra_pathogen(ppi_df, pathogen_taxonids)
# assert all(temp_intra_species == ppi_df['inter-intra-species'])
# assert all(temp_intra_pathogen == ppi_df['inter-intra-pathogen'])
ppi_df = ppi_df.loc[(ppi_df['inter-intra-species'] != 'intra')
                    & (ppi_df['inter-intra-pathogen'] != 'intra')]
ppi_df = ppi_df.reset_index(drop=True)
ppi_df.drop(
    ['inter-intra-species', 'inter-intra-pathogen'], axis=1, inplace=True)
print('All intra-species and intra-pathogen interactions were omitted.')
# check for intra-hosts cross species
assert any(
    ppi_df.taxid_A.isin(host_taxonids) & ppi_df.taxid_B.isin(host_taxonids)
) is False
# create unique identifier by combining xrefs
ppi_filter.unique_identifier(ppi_df)
ppi_df['original_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])

# fix bad xref identifiers (i.e. containing more than a single entry)
id_mapper.check_unique_identifier(ppi_df)

# re-create unique identifier by combining xrefs
ppi_filter.unique_identifier(ppi_df)
ppi_df['checker_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])
print(
    'By removing/fixing protein identifiers that consisted of multiple entries, the following duplicates were introduced in the dataset:\n'
)
print(ppi_df.loc[(ppi_df.original_unique)
                 & ~(ppi_df.checker_unique)].groupby('origin').size(),'\n')

# create UniProt mapping files and remap identifiers
out_mappings = out_path / 'mapping'
out_mappings.parent.mkdir(parents=True, exist_ok=True)
id_mapper.map2uniprot(ppi_df, out_mappings, reviewed_only=True)

# remove multiple mappings
ppi_df = id_mapper.remove_mult(ppi_df)

# re-create unique identifier by combining xrefs
ppi_filter.unique_identifier(ppi_df)
ppi_df['remap_unique'] = ~ppi_df.duplicated(subset=['xref_partners_sorted'])
print(
    'The act of remapping protein identifiers to UniProt AC, introduced the following duplicates:\n'
)
print(ppi_df.loc[(ppi_df.checker_unique)
                 & ~(ppi_df.remap_unique)].groupby('origin').size(),'\n')

# check if remapping creates new unique values (should never happen)
assert ppi_df.loc[
    ~(ppi_df.original_unique) & (ppi_df.remap_unique), [
        'xref_partners_sorted', 'original_unique', 'remap_unique',
        'checker_unique'
    ]].shape[0] == 0

# report information about duplicates
print('Number of unique interactions per raw dataset:')
print(ppi_df.groupby('origin')['xref_partners_sorted'].nunique(),'\n')
print('Total dataset size:')
print(ppi_df.groupby('origin')['xref_partners_sorted'].size(),'\n')
print('Total number of unique interactions out of {}'.format(
    ppi_df.shape[0]))
print(np.sum(~ppi_df.duplicated(subset=['xref_partners_sorted'])),'\n')
print(
    'Total number of unique interactions out of {}, where publications are considered unique:'.
    format(ppi_df.shape[0]))
print(
    np.sum(~ppi_df.duplicated(subset=['xref_partners_sorted', 'publication'])),'\n')
ppi_df.drop(
    ['original_unique', 'checker_unique', 'remap_unique'],
    axis=1,
    inplace=True)

TaxonIDs were read from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/taxonid/10292-child-taxids.txt and /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/taxonid/interacting-taxids.txt

taxid:9534
All intra-species and intra-pathogen interactions were omitted.
26273 unique identifiers in column xref_A did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

26273 unique identifiers in column xref_B did not conform to the PSI-MITAB format (multiple identifiers were present) and a fix was attempted.

By removing/fixing protein identifiers that consisted of multiple entries, the following duplicates were introduced in the dataset:

origin
hpidb2-19.02.2018.mitab    298
phi_data.csv                 8
dtype: int64 


No ddbj/embl/genbank: identifiers found in dataset.

No ensembl: identifiers found in dataset.

No ensemblgenomes: identifiers found in da

In [26]:
ppi_df

,aliases_A,aliases_B,alt_identifiers_A,alt_identifiers_B,author,confidence_score,detection_method,display_id_A,display_id_B,interaction_identifiers,...,pathogen,publication,source_database_ids,taxid_A,taxid_B,xref_A,xref_B,taxid_A_name,taxid_B_name,xref_partners_sorted
0,-,entrez gene/locuslink:ALS14(gene name synonym)...,biogrid:971433|entrez gene/locuslink:US2|entre...,biogrid:113258|entrez gene/locuslink:VCP,Chevalier MS (2003),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:426823,...,NaN,pubmed:12663780,"psi-mi:""MI:0463""(biogrid)",taxid:10298,taxid:9606,uniprotkb:P06485,uniprotkb:P55072,Human alphaherpesvirus 1,Homo sapiens,uniprotkb:P06485%uniprotkb:P55072
1,-,entrez gene/locuslink:ALS14(gene name synonym)...,biogrid:971434|entrez gene/locuslink:US3|entre...,biogrid:113258|entrez gene/locuslink:VCP,Chevalier MS (2003),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:426824,...,NaN,pubmed:12663780,"psi-mi:""MI:0463""(biogrid)",taxid:10298,taxid:9606,uniprotkb:P04413,uniprotkb:P55072,Human alphaherpesvirus 1,Homo sapiens,uniprotkb:P04413%uniprotkb:P55072
2,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971457|entrez gene/locuslink:UL9|entre...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430623,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,entrez gene/locuslink:2703434,Homo sapiens,Human alphaherpesvirus 1,entrez gene/locuslink:2703434%uniprotkb:P51610
3,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971473|entrez gene/locuslink:UL30|entr...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430624,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,uniprotkb:P04293,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P04293%uniprotkb:P51610
4,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971449|entrez gene/locuslink:UL52|entr...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430625,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,uniprotkb:P10236,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P10236%uniprotkb:P51610
5,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971457|entrez gene/locuslink:UL9|entre...,Peng H (2010),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:430626,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,entrez gene/locuslink:2703434,Homo sapiens,Human alphaherpesvirus 1,entrez gene/locuslink:2703434%uniprotkb:P51610
6,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971473|entrez gene/locuslink:UL30|entr...,Peng H (2010),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:430627,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,uniprotkb:P04293,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P04293%uniprotkb:P51610
7,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971449|entrez gene/locuslink:UL52|entr...,Peng H (2010),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:430628,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,uniprotkb:P10236,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P10236%uniprotkb:P51610
8,-,entrez gene/locuslink:CIA-II(gene name synonym),biogrid:971473|entrez gene/locuslink:UL30|entr...,biogrid:120845|entrez gene/locuslink:ASF1B,Peng H (2010),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:430633,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:10298,taxid:9606,uniprotkb:P04293,uniprotkb:Q9NVP2,

In [33]:
ppi_df.groupby('origin').size()

origin
BIOGRID-ALL-3.4.160.mitab        1168
hpidb2-19.02.2018.mitab          8942
intact-virus-22.03.2018.mitab    3288
phi_data.csv                     8118
virhostnet-01.2018.mitab         4328
dtype: int64

In [32]:
ppi_df.loc[ppi_df['taxid_A'].isin(host_taxonids)].groupby('origin').size()

origin
BIOGRID-ALL-3.4.160.mitab         226
hpidb2-19.02.2018.mitab          8942
intact-virus-22.03.2018.mitab     439
phi_data.csv                     8118
virhostnet-01.2018.mitab         1379
dtype: int64

In [31]:
ppi_df.loc[ppi_df['taxid_B'].isin(host_taxonids)].groupby('origin').size()

origin
BIOGRID-ALL-3.4.160.mitab         942
intact-virus-22.03.2018.mitab    2849
virhostnet-01.2018.mitab         2949
dtype: int64

In [35]:
ppi_df.loc[ppi_df['taxid_A'].isin(pathogen_taxonids)].groupby('origin').size()

origin
BIOGRID-ALL-3.4.160.mitab         942
intact-virus-22.03.2018.mitab    2849
virhostnet-01.2018.mitab         2949
dtype: int64

In [36]:
ppi_df.loc[ppi_df['taxid_B'].isin(pathogen_taxonids)].groupby('origin').size()

origin
BIOGRID-ALL-3.4.160.mitab         226
hpidb2-19.02.2018.mitab          8942
intact-virus-22.03.2018.mitab     439
phi_data.csv                     8118
virhostnet-01.2018.mitab         1379
dtype: int64

In [38]:
ppi_df.loc[ppi_df['taxid_B'].isin(pathogen_taxonids)].shape

(19104, 22)

In [39]:
ppi_df.loc[ppi_df['taxid_A'].isin(pathogen_taxonids)].shape

(6740, 22)

In [44]:
ppi_df.columns

Index(['aliases_A', 'aliases_B', 'alt_identifiers_A', 'alt_identifiers_B',
       'author', 'confidence_score', 'detection_method', 'display_id_A',
       'display_id_B', 'interaction_identifiers', 'interaction_type', 'origin',
       'pathogen', 'publication', 'source_database_ids', 'taxid_A', 'taxid_B',
       'xref_A', 'xref_B', 'taxid_name_A', 'taxid_name_B',
       'xref_partners_sorted'],
      dtype='object')

In [67]:
host_list = set(host_taxonids)
host_position_mask = ppi_df['taxid_B'].isin(host_list)
column_names = ['xref', 'taxid', 'aliases', 'alt_identifiers', 'display_id', 'taxid_name']
columns_to_swap = [name + label for name in column_names for label in ['_A', '_B']]
columns_after_swap = [name + label for name in column_names for label in ['_B', '_A']]
ppi_df.loc[host_position_mask, columns_to_swap] = ppi_df.loc[host_position_mask, columns_after_swap].values


In [68]:
ppi_df.loc[0:3]

,aliases_A,aliases_B,alt_identifiers_A,alt_identifiers_B,author,confidence_score,detection_method,display_id_A,display_id_B,interaction_identifiers,...,pathogen,publication,source_database_ids,taxid_A,taxid_B,xref_A,xref_B,taxid_name_A,taxid_name_B,xref_partners_sorted
0,entrez gene/locuslink:ALS14(gene name synonym)...,-,biogrid:113258|entrez gene/locuslink:VCP,biogrid:971433|entrez gene/locuslink:US2|entre...,Chevalier MS (2003),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:426823,...,NaN,pubmed:12663780,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P55072,uniprotkb:P06485,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P06485%uniprotkb:P55072
1,entrez gene/locuslink:ALS14(gene name synonym)...,-,biogrid:113258|entrez gene/locuslink:VCP,biogrid:971434|entrez gene/locuslink:US3|entre...,Chevalier MS (2003),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:426824,...,NaN,pubmed:12663780,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P55072,uniprotkb:P04413,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P04413%uniprotkb:P55072
2,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971457|entrez gene/locuslink:UL9|entre...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430623,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,entrez gene/locuslink:2703434,Homo sapiens,Human alphaherpesvirus 1,entrez gene/locuslink:2703434%uniprotkb:P51610
3,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971473|entrez gene/locuslink:UL30|entr...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430624,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,uniprotkb:P04293,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P04293%uniprotkb:P51610


In [66]:
ppi_df.loc[0:3]

,aliases_A,aliases_B,alt_identifiers_A,alt_identifiers_B,author,confidence_score,detection_method,display_id_A,display_id_B,interaction_identifiers,...,pathogen,publication,source_database_ids,taxid_A,taxid_B,xref_A,xref_B,taxid_name_A,taxid_name_B,xref_partners_sorted
0,-,entrez gene/locuslink:ALS14(gene name synonym)...,biogrid:971433|entrez gene/locuslink:US2|entre...,biogrid:113258|entrez gene/locuslink:VCP,Chevalier MS (2003),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:426823,...,NaN,pubmed:12663780,"psi-mi:""MI:0463""(biogrid)",taxid:10298,taxid:9606,uniprotkb:P06485,uniprotkb:P55072,Human alphaherpesvirus 1,Homo sapiens,uniprotkb:P06485%uniprotkb:P55072
1,-,entrez gene/locuslink:ALS14(gene name synonym)...,biogrid:971434|entrez gene/locuslink:US3|entre...,biogrid:113258|entrez gene/locuslink:VCP,Chevalier MS (2003),-,"psi-mi:""MI:0004""(affinity chromatography techn...",NaN,NaN,biogrid:426824,...,NaN,pubmed:12663780,"psi-mi:""MI:0463""(biogrid)",taxid:10298,taxid:9606,uniprotkb:P04413,uniprotkb:P55072,Human alphaherpesvirus 1,Homo sapiens,uniprotkb:P04413%uniprotkb:P55072
2,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971457|entrez gene/locuslink:UL9|entre...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430623,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,entrez gene/locuslink:2703434,Homo sapiens,Human alphaherpesvirus 1,entrez gene/locuslink:2703434%uniprotkb:P51610
3,entrez gene/locuslink:CFF(gene name synonym)|e...,-,biogrid:109304|entrez gene/locuslink:HCFC1,biogrid:971473|entrez gene/locuslink:UL30|entr...,Peng H (2010),-,"psi-mi:""MI:0018""(two hybrid)",NaN,NaN,biogrid:430624,...,NaN,pubmed:20133788,"psi-mi:""MI:0463""(biogrid)",taxid:9606,taxid:10298,uniprotkb:P51610,uniprotkb:P04293,Homo sapiens,Human alphaherpesvirus 1,uniprotkb:P04293%uniprotkb:P51610


In [63]:
ppi_df.loc[host_position_mask, columns_after_swap].values

array([['uniprotkb:P55072', 'uniprotkb:P06485', 'taxid:9606', ..., nan,
        'Homo sapiens', 'Human alphaherpesvirus 1'],
       ['uniprotkb:P55072', 'uniprotkb:P04413', 'taxid:9606', ..., nan,
        'Homo sapiens', 'Human alphaherpesvirus 1'],
       ['uniprotkb:Q9NVP2', 'uniprotkb:P04293', 'taxid:9606', ..., nan,
        'Homo sapiens', 'Human alphaherpesvirus 1'],
       ...,
       ['uniprotkb:Q9UGN5', 'uniprotkb:Q9QR71', 'taxid:9606', ..., nan,
        'Homo sapiens', 'Human herpesvirus 8 strain GK18'],
       ['uniprotkb:Q9Y2J2', 'uniprotkb:Q9QR71', 'taxid:9606', ..., nan,
        'Homo sapiens', 'Human herpesvirus 8 strain GK18'],
       ['uniprotkb:Q9Y4C1', 'uniprotkb:Q9QR71', 'taxid:9606', ..., nan,
        'Homo sapiens', 'Human herpesvirus 8 strain GK18']], dtype=object)

In [ ]:
    host_list = set(host_list)
    host_position_mask = interaction_dataframe['taxid_B'].isin(host_list)
    column_names = ['xref', 'taxid', 'aliases', 'alt_identifiers', 'display_id', 'taxid_name']
    columns_to_swap = [name + label for name in column_names for label in ['_A', '_B']]
    columns_after_swap = [name + label for name in column_names for label in ['_B', '_A']]
    interaction_dataframe.loc[host_position_mask, columns_to_swap] = interaction_dataframe.loc[host_position_mask, columns_after_swap].values


In [77]:
all_identifiers = pd.unique(interaction_dataframe[columns].values.ravel('K'))
out_identifiers = Path('/media/pieter/Seagate Red Pieter Moris/workdir/uniprot_identifiers.txt')
with out_identifiers.open('w') as out:
    for i in all_identifiers:
        out.write('{}\n'.format(i))